<a href="https://colab.research.google.com/github/Exion007/Colab/blob/main/multiclass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import pandas as pd
import re

from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

In [34]:
rawdf = pd.read_csv('merged_ls_200923.csv', engine="python", on_bad_lines="skip")
rawdf.dropna(inplace=True)

rawdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21210 entries, 0 to 23667
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    21210 non-null  object
 1   label   21210 non-null  object
dtypes: object(2)
memory usage: 497.1+ KB


In [35]:
labels = ['AVI', 'MIS', 'OTH', 'WIN', 'WEA',
           'COC', 'PRP', 'MEC', 'ELC', 'LAG',
           'FLC', 'FFC', 'ECS', 'FLU']

def filter_by_label(df):
  filtered_df = rawdf[rawdf['label'].apply(lambda x: isinstance(x, str) and x in labels)]
  return filtered_df

def expand_contractions(text):
    contractions = {
        "n't": "not",
        "'ve": "have",
        "'ll": "will",
        "'d": "would",
        "'re": "are",
        "'s": "is",
        "'m": "am"
    }
    pattern = re.compile(r"\b(?:" + "|".join(contractions.keys()) + r")\b")
    return pattern.sub(lambda match: contractions[match.group(0)], text)


def remove_special_characters(text):
    special_characters_pattern = re.compile(r"[#%@\<>{}()=\[\]*\-\\|_½]")
    return special_characters_pattern.sub("", text.lower())

df = filter_by_label(rawdf)
new = df["text"].apply(expand_contractions)
df["text"] = new
new = df["text"].apply(remove_special_characters)
df["text"] = new
df.head()

<ipython-input-35-d85f75270db5>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"] = new
<ipython-input-35-d85f75270db5>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"] = new


,text,label
0,the pentagon is upgrading its gps sniper atp ...,AVI
1,the search continues for a japanese air selfde...,MIS
2,some recent photos of the hill afb f35s deploy...,COC
3,kfx: south korea wants to build its very own f...,MIS
4,the army aviation and missile command awarded ...,AVI


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20867 entries, 0 to 23667
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    20867 non-null  object
 1   label   20867 non-null  object
dtypes: object(2)
memory usage: 489.1+ KB


In [37]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

max_vocab_size = 10000
tokenizer = Tokenizer(num_words=max_vocab_size)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)


max_seq_length = 100
X_train_padded = pad_sequences(X_train_seq, maxlen=max_seq_length)
X_test_padded = pad_sequences(X_test_seq, maxlen=max_seq_length)

In [38]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [39]:
embedding_dim = 100
num_classes = len(df['label'].unique())

In [40]:
model = Sequential()
model.add(Embedding(input_dim=max_vocab_size, output_dim=embedding_dim, input_length=max_seq_length))
model.add(LSTM(128))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

batch_size = 32
num_epochs = 10
model.fit(X_train_padded, y_train_encoded, epochs=num_epochs, batch_size=batch_size, validation_data=(X_test_padded, y_test_encoded))

#y_pred_encoded = model.predict_classes(X_test_padded)

Epoch 1/10
522/522 [==============================] - 111s 206ms/step - loss: 1.6959 - accuracy: 0.3987 - val_loss: 1.5929 - val_accuracy: 0.4149
Epoch 2/10
522/522 [==============================] - 105s 201ms/step - loss: 1.4774 - accuracy: 0.4722 - val_loss: 1.5409 - val_accuracy: 0.4425
Epoch 3/10
522/522 [==============================] - 103s 197ms/step - loss: 1.2772 - accuracy: 0.5465 - val_loss: 1.5744 - val_accuracy: 0.4382
Epoch 4/10
522/522 [==============================] - 105s 202ms/step - loss: 1.1080 - accuracy: 0.6085 - val_loss: 1.7168 - val_accuracy: 0.4418
Epoch 5/10
522/522 [==============================] - 102s 196ms/step - loss: 0.9640 - accuracy: 0.6596 - val_loss: 1.7428 - val_accuracy: 0.4487
Epoch 6/10
522/522 [==============================] - 103s 198ms/step - loss: 0.8461 - accuracy: 0.7043 - val_loss: 1.9025 - val_accuracy: 0.4483
Epoch 7/10
522/522 [==============================] - 106s 202ms/step - loss: 0.7482 - accuracy: 0.7328 - val_loss: 1.9662 -

AttributeError: ignored

In [46]:
import numpy as np
y_pred = model.predict(X_test_padded)
y_pred_encoded = np.argmax(y_pred,axis=1)

131/131 [==============================] - 6s 47ms/step


In [47]:
accuracy = accuracy_score(y_test_encoded, y_pred_encoded)
print("Accuracy:", accuracy)
print(classification_report(y_test_encoded, y_pred_encoded))

Accuracy: 0.44825107810253956
              precision    recall  f1-score   support

           0       0.54      0.53      0.53      1531
           1       0.28      0.17      0.21       376
           2       0.00      0.00      0.00         8
           3       0.00      0.00      0.00         5
           4       0.28      0.24      0.26        29
           5       0.34      0.37      0.35       103
           6       0.00      0.00      0.00         3
           7       0.07      0.03      0.04        39
           8       0.00      0.00      0.00        23
           9       0.38      0.44      0.41       824
          10       0.50      0.55      0.53       824
          11       0.27      0.29      0.28        73
          12       0.37      0.35      0.36       316
          13       0.20      0.05      0.08        20

    accuracy                           0.45      4174
   macro avg       0.23      0.22      0.22      4174
weighted avg       0.44      0.45      0.44      4